# Random Forest Rule Extraction

## Imports

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.tree import _tree, export_text
from sklearn.ensemble import RandomForestClassifier

## (Assumed) Build Random Forest Model
Here it is assumed you have a Random Forest Model (named best_rf) pre-built and fitted on the training set

In [ ]:
# Fitting the model with the best parameters on the whole training set
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)

## Store Rules and Visualize the Trees

In [ ]:
# Initializing an empty list to store rules from decision trees
rules = []

# Iterating through each decision tree in the random forest
for tree in best_rf.estimators_:
    # Exporting the structure of the decision tree as text and appending it to the rules list
    r = export_text(tree, feature_names=list(X.columns), show_weights=True, max_depth=10)
    rules.append(r)

# Printing rules of the first 5 decision trees
for rule in rules[:5]:
    print(rule)

In [12]:
# Example output

Random Forest Trees Structure: 

|--- Feature 1 <= 0.50
|   |--- Feature 2 <= 790.61
|   |   |--- Feature 3 <= 0.50
|   |   |   |--- Feature 4 <= 19.39
|   |   |   |   |--- Feature 5 <= 811.19
|   |   |   |   |   |--- Feature 6 <= 1.50
|   |   |   |   |   |   |--- Feature 7 <= 0.01
|   |   |   |   |   |   |   |--- Feature 8 <= 1389.50
|   |   |   |   |   |   |   |   |--- Feature 9 <= 0.50
|   |   |   |   |   |   |   |   |   |--- Feature 10 <= 6.50
|   |   |   |   |   |   |   |   |   |   |--- weights: [5825.00, 39.00] class: 0.0
|   |   |   |   |   |   |   |   |   |--- Feature 10 >  6.50
|   |   |   |   |   |   |   |   |   |   |--- weights: [2923.00, 80.00] class: 0.0
|   |   |   |   |   |   |   |   |--- Feature 9 >  0.50
|   |   |   |   |   |   |   |   |   |--- Feature 11 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- weights: [293.00, 27.00] class: 0.0
|   |   |   |   |   |   |   |   |   |--- Feature 11 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- weights: [191.00, 47.00

## Code to Extract Rules from all Random Forest Trees

In [ ]:
# Defining function to extract rules from a decision tree
def get_rules(tree, feature_names):
    tree_ = tree.tree_  # Getting the underlying tree structure
    # Creating a list of feature names, replacing undefined features with a placeholder string
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []  # List to store all the paths (rules) in the tree
    path = []  # Temporary list to store the current path

    # Recursive function to traverse the tree and collect rules
    def recurse(node, path, paths):
        # If the node is not a leaf node
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 6)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 6)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            # If the node is a leaf node, append the current path to paths
            path += [(tree_.value[node], np.sum(tree_.value[node]))]
            paths += [path]

    # Starting the recursive function from the root node
    recurse(0, path, paths)

    # Sorting the paths based on the number of samples
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []  # List to store the rules in a formatted way
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        
        classes = path[-1][0][0]
        l = np.argmax(classes)
        class_label = l
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        samples = path[-1][1]
        
        # Appending the formatted rule to the rules list
        rules.append({
            'rule': rule,
            'class': class_label,
            'samples': samples,
            'proba': proba
        })
        
    return rules

# Getting feature names from the DataFrame df_train, excluding the target variable column
feature_names = df_train.columns[1:]  

# Creating an empty list to store the rules from each tree
rules_list = []

# Iterating through each decision tree in the random forest to collect rules
for tree in best_rf.estimators_:
    rules = get_rules(tree, feature_names)
    rules_list.extend(rules)

# Converting the list of dictionaries into a DataFrame
rules_df = pd.DataFrame(rules_list)

# Renaming columns for better interpretation
rules_df.rename(columns={"rule": "Feature Rule"}, inplace=True)
rules_df = rules_df[rules_df["class"] == 1].copy()
rules_df.drop("class", axis=1, inplace=True)
rules_df.rename(columns={"samples": "Total Samples"}, inplace=True)
rules_df.rename(columns={"proba": "Probability of X"}, inplace=True)
rules_df.reset_index(drop=True, inplace=True)

# Displaying the first 5 rows of the rules DataFrame
rules_df.head()

In [7]:
# Example DataFrame output of rules_df

,Feature Rule,Total_Samples,Fraud_Probability
0,(Feature_1 > 0.004) and (Feature_2 <= 250.25) and (Feature_3 > 2.5) and (Feature_4 <= 400.2001) and (Feature_5 <= 0.050) and (Feature_6 <= 1.0) and (Feature_7 > 1.0) and (Feature_8 <= 159.75) and (Feature_1 > 0.07256),361,0.89845
1,(Feature_9 <= 1.0) and (Feature_10 <= 391.162491) and (Feature_11 > 1.0) and (Feature_12 <= 41.75),290,0.79862
2,(Feature_13 > 5.0) and (Feature_9 <= 1.0) and (Feature_11 > 1.0) and (Feature_14 > 7.0) and (Feature_15 <= 1249.25) and (Feature_16 > 1.0) and (Feature_17 <= 5.0) and (Feature_18 <= 60.0) and (Feature_19 <= 7.0) and (Feature_14 > 17.0),248,0.79516
3,(Feature_9 <= 1.0) and (Feature_11 > 1.0) and (Feature_20 <= 16.25) and (Feature_21 <= 881.75) and (Feature_22 <= 3.75) and (Feature_23 <= 179479.3828125) and (Feature_24 <= 103.6275025) and (Feature_25 > 780.0650025),302,0.78674


## Print and Export the Rules

In [9]:
# Print best rules
print('Feature Rules:', "\n")
pd.set_option('display.max_colwidth', None)
for item in rules_df['Feature Rule'][0:20]:
    print(item, "\n")
    
# Export best rules
rules_df.to_csv('Random Forest Rules.csv', index=False)

Feature Rules: 

(Feature_1 > 0.004) and (Feature_2 <= 250.25) and (Feature_3 > 2.5) and (Feature_4 <= 400.2001) and (Feature_5 <= 0.050) and (Feature_6 <= 1.0) and (Feature_7 > 1.0) and (Feature_8 <= 159.75) and (Feature_1 > 0.07256) 

(Feature_9 <= 1.0) and (Feature_10 <= 391.162491) and (Feature_11 > 1.0) and (Feature_12 <= 41.75) 

(Feature_13 > 5.0) and (Feature_9 <= 1.0) and (Feature_11 > 1.0) and (Feature_14 > 7.0) and (Feature_15 <= 1249.25) and (Feature_16 > 1.0) and (Feature_17 <= 5.0) and (Feature_18 <= 60.0) and (Feature_19 <= 7.0) and (Feature_14 > 17.0) 

(Feature_9 <= 1.0) and (Feature_11 > 1.0) and (Feature_20 <= 16.25) and (Feature_21 <= 881.75) and (Feature_22 <= 3.75) and (Feature_23 <= 179479.3828125) and (Feature_24 <= 103.6275025) and (Feature_25 > 780.0650025) 

